**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]

.

.


# Problem Solving by Searching
---

## Why search?

> Search techniques are utilized in problem-solving scenarios when there is a clear, predefined goal that needs to be achieved, and the problem can be logically broken down into different states and actions.

Search algorithms are essential for tackling problems in various domains and are especially useful in cases when:
- The problem's **complexity** is beyond straightforward mathematical modeling or doesn't have a known analytical solution.
- We **lack prior knowledge** of the exact sequence of actions leading to a solution.
- We intend to systematically explore a **large solution space**, requiring an examination of various paths to find the best one.
- We seek **optimal or near-optimal solutions** in extensive search spaces.

### Problem solving agent
A problem-solving agent is a type of goal-based agent capable of determining a sequence of actions leading to a desired goal. This agent achieves this by exploring the problem's state space, which encompasses all potential states reachable from the initial state through a sequence of actions.

The agent's program comprises two key components:
- **Problem & Goal Formulation** This entails the process of deciding which actions and states to consider based on the specified goal. 🗺️🎯..
- **Search Algorithm** This involves determining the next action to take given the current state in order to progress toward the goal. 🧭..

### Conditions for solving problems by searching
While path search algorithms can be applied to a wide range of problems, there are some conditions and considerations that make them more suitable for certain types of problems.

When the task environment is
- discrete
- fully observable
- deterministic
- static
- completelly known

> In this case, the agent can be said to have `full control` of the environment, therefore it can plan ahead and choose a sequence of actions that will lead to the desired goal. The process of finding a sequence of actions is called *search*.

## Problem definition

A problem can be formally defined by the following components:
1. **Initial state** The state in which the agent begins. $s_0$
2. **Set of actions** The set of actions available to the agent in each state. $\mathcal{A}(s)=\{a_1, a_2, \dots, a_n\}$
3. **Transition model** A description of what each action does. $\mathcal{S}(s,a)=s'$
4. **Goal test** A function that determines whether a given state is a goal state. $\mathcal{G}(s)$
5. **Path cost** A function that assigns a numeric cost to each path. It is assumed that the cost of a path is the sum of the costs of its actions, i.e. the sum of each step's cost. $g(s_0, a_1, \dots, a_n)=\sum_{i=1}^n c(s_i, a_i, s_{i+1})$

### State space
The state space is the set of all states reachable from the initial state by any sequence of actions. It is denoted by $\mathcal{S}$. The state space is a **directed graph**, where 
- the nodes represent `states` ⭕..
- the edges represent `actions` ➡️..

### Searching for solutions
A solution to a problem is a sequence of actions that leads *from the initial state to a goal state*. A solution is `optimal` if it has the lowest path cost among all solutions.

Depending on the way we want our search algorithm to explore the state space, we can distinguish between two types of search.

##### 1 - 🌳 Tree search 🌳
In a tree search, the search algorithm explores a search tree without considering whether it has visited a state before. It doesn't keep track of the states it has already explored.

$\implies$ different nodes can represent the same state.

The initial state of the problem serves as the `root` node of the tree. As the search algorithm progresses, it `expands nodes` by considering possible actions and generating child nodes for each state that can be reached.
- `o` `Memory usage`: It tends to use less memory because it only maintains the current tree and doesn't remember visited states.
- `x` `Redundancy`: It may explore the same state multiple times, leading to inefficiency.
- `x` `Un-completeness`: It may not find a solution even if one exists, due to infinite loops in a space with cycles.

Tree search is typically used in cases where memory resources are limited and revisiting states isn't an issue. It's suitable for cases where the solution is guaranteed to be found quickly or where the state space doesn't contain cycles.
  
##### 2 - 📊 Graph search 📊
In graph search, the search algorithm keeps track of the states it has already explored, ensuring that it doesn't revisit them. This is crucial for practical problem-solving because revisiting states can be inefficient and may lead to infinite loops.

$\implies$ nodes represent unique states.

- `x` `Memory usage`: It tends to use more memory because it needs an additional data structure to keep track of visited states.
- `o` `Efficient exploration`: In graph search, the exploration extends across the entire state space, keeping track of visited states to avoid revisiting them. It maintains a more comprehensive record of the explored states.
- `o` `Completeness`: It is guaranteed to find a solution if one exists, as it doesn't get stuck in infinite loops.

Graph search is used when memory resources are more abundant, and the state space may contain cycles or when it's essential to guarantee that a solution, if it exists, will be found.

## Data structures & common functions

## Data structures for search



So, what is the key difference between tree and graph search? The answer lies in the data structure used to store the nodes.

- frontier
- explored set
    

State Space Representation:

Graph: In graph search, you explicitly represent the state space as a graph data structure. Nodes are the states, and edges are the actions that transition between states.
Tree: In tree search, you represent the state space as a tree, which is an abstract structure without duplicated states. It's created as the search progresses from the initial state.

Frontier (Open Set):

Queue (FIFO) for Breadth-First Search (BFS): In BFS, a queue is used to store nodes in the order they were added. The first node added (the root) is the first to be expanded.
Stack (LIFO) for Depth-First Search (DFS): In DFS, a stack is employed to store nodes. The most recently added node is expanded first.

Explored Set (Closed Set):

Set or List: To keep track of nodes that have already been explored to avoid revisiting them. These data structures help prevent loops in the search.
    - frontier as separator between explored and unexplored states

Actions ??????

Path and Node Representation: ????

Node Data Structure: Each node in the search tree or graph typically contains information such as the current state, the action taken to reach that state, the parent node, and the cost associated with reaching that state.
Path: A list or stack to store the path from the initial state to the current state during the search.

## Search strategies

### Measuring search performance

### Uninformed search strategies

#### Breadth-first search (BFS)

#### Depth-first search (DFS)

#### Uniform-cost search (UCS)

#### Depth-limited search (DLS)

#### Iterative deepening search (IDS)

#### Bidirectional search (BDS)

#### Beam search

### Informed search strategies

#### Greedy best-first search (GBFS)

#### A* search


- search tree vs search graph: data structures
  => treat states and actions as **atomic** entities

## Search strategies
Their performance is measured in terms of:
- Completeness: is the algorithm guaranteed to find a solution when there is one?
- Optimality: does the strategy find the best solution?
- Time complexity: how long does it take to find a solution?
- Space complexity: how much memory is needed to perform the search?
Complexity depends on:
- b: maximum branching factor of the search tree
- d: depth of the least-cost solution

### Uninformed search strategies
- Breadth-first search
- Uniform-cost search
- Depth-first search
- Depth-limited search
- Iterative deepening search
- Bidirectional search
- Beam search

### Informed search strategies
- Greedy best-first search
- A* search
- Recursive best-first search (RBFS)
- Simplified Memory-bounded A* search (SMA*)



